# Create the SQLite Database

In [23]:
import sqlite3

# Create a new SQLite database 
conn = sqlite3.connect('company.db')
cursor = conn.cursor()

# Create Employees table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Employees (
        ID INTEGER PRIMARY KEY,
        Name TEXT NOT NULL,
        Department TEXT NOT NULL,
        Salary INTEGER NOT NULL,
        Hire_Date TEXT NOT NULL
    )
''')

# Create Departments table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Departments (
        ID INTEGER PRIMARY KEY,
        Name TEXT NOT NULL,
        Manager TEXT NOT NULL
    )
''')


In [24]:
# Insert  data into Employees table
cursor.executemany('''
    INSERT INTO Employees (ID, Name, Department, Salary, Hire_Date)
    VALUES (?, ?, ?, ?, ?)
''', [
    (1, 'Alice', 'Sales', 50000, '2021-01-15'),
    (2, 'Bob', 'Engineering', 70000, '2020-06-10'),
    (3, 'Charlie', 'Marketing', 60000, '2022-03-20')
])


In [25]:
# Insert data into Departments table
cursor.executemany('''
    INSERT INTO Departments (ID, Name, Manager)
    VALUES (?, ?, ?)
''', [
    (1, 'Sales', 'Alice'),
    (2, 'Engineering', 'Bob'),
    (3, 'Marketing', 'Charlie')
])

In [26]:
# Commit changes and close the connection
conn.commit()
conn.close()

# Implement Chat assistant 

In [27]:
import sqlite3
import re
from datetime import datetime

In [28]:
# Connect to SQLite database
def get_db_connection():
    conn = sqlite3.connect('company.db')
    return conn

In [30]:
# Handle user queries
def handle_query(query):
    conn = get_db_connection()
    cursor = conn.cursor()

    # Query 1: Show all employees in a department
    match = re.match(r"show me all employees in the (.+) department", query, re.I)
    if match:
        department = match.group(1).capitalize()
        cursor.execute("SELECT * FROM Employees WHERE Department=?", (department,))
        employees = cursor.fetchall()
        if employees:
            return format_employees(employees)
        else:
            return f"No employees found in the {department} department."

    # Query 2: Who is the manager of a department?
    match = re.match(r"who is the manager of the (.+) department", query, re.I)
    if match:
        department = match.group(1).capitalize()
        cursor.execute("SELECT Manager FROM Departments WHERE Name=?", (department,))
        manager = cursor.fetchone()
        if manager:
            return f"The manager of the {department} department is {manager[0]}."
        else:
            return f"No department named {department} found."

    # Query 3: List employees hired after a specific date
    match = re.match(r"list all employees hired after (.+)", query, re.I)
    if match:
        try:
            hire_date = datetime.strptime(match.group(1), "%Y-%m-%d")
            cursor.execute("SELECT * FROM Employees WHERE Hire_Date > ?", (hire_date.strftime('%Y-%m-%d'),))
            employees = cursor.fetchall()
            if employees:
                return format_employees(employees)
            else:
                return "No employees found hired after that date."
        except ValueError:
            return "Please enter the date in the format YYYY-MM-DD."

    # Query 4: Total salary expense for a department
    match = re.match(r"what is the total salary expense for the (.+) department", query, re.I)
    if match:
        department = match.group(1).capitalize()
        cursor.execute("SELECT SUM(Salary) FROM Employees WHERE Department=?", (department,))
        total_salary = cursor.fetchone()[0]
        if total_salary is not None:
            return f"The total salary expense for the {department} department is ${total_salary}."
        else:
            return f"No employees found in the {department} department."

    # If no pattern matches
    return "Sorry, I didn't understand that query. Could you please rephrase?"

# Format employees for easy readability
def format_employees(employees):
    formatted = "ID | Name | Department | Salary | Hire Date\n"
    formatted += "-" * 50 + "\n"
    for emp in employees:
        formatted += f"{emp[0]} | {emp[1]} | {emp[2]} | ${emp[3]} | {emp[4]}\n"
    return formatted

# Main chat loop
def chat():
    print("Hello! I am your chat assistant. How can I help you today?")
    while True:
        query = input("You: ")
        if query.lower() in ['exit', 'quit', 'bye']:
            print("Goodbye!")
            break
        response = handle_query(query)
        print("Assistant: " + response)

# Run the chat assistant
if __name__ == '__main__':
    chat()

Hello! I am your chat assistant. How can I help you today?


You:  "Show me all employees in the Marketing department."


Assistant: Sorry, I didn't understand that query. Could you please rephrase?


You:  Show me all employees in the Marketing department.


Assistant: ID | Name | Department | Salary | Hire Date
--------------------------------------------------
3 | Charlie | Marketing | $60000 | 2022-03-20



...

Assistant: The total salary expense for the Marketing department is $60000.


You:  What is the total salary expense for the Engineering department?


Assistant: The total salary expense for the Engineering department is $70000.


You:  exit


Goodbye!
